# Описание Домашнего Задания

Нейросети – ML - модель

<b>Цель:</b>
В данном домашнем задании вы потренируетесь в построении модели машинного обучения для формирования вашей торговой стратегии на основе нейросетей.


<b>Описание/Пошаговая инструкция выполнения домашнего задания:</b>
Уважаемый слушатель!


Вы успешно создали полноценную торговую стратегию на основе модели машинного обучения и обеспечили фиксацию и сравнение метрик полученных моделей.


Вы решаете построить более сложные модели машинного обучения и хотите использовать нейросетевые модели, в том числе для обработки временных рядов и глубокие нейронные сети. Здесь вам могут помочь методы рекуррентных сетей и современные трансформерные архитектуры.


Поговорив с коллегами, вы понимаете, что самостоятельно построить и обучить действительно сложные архитектуры будет сложно и решаете воспользоваться предобученными свободно распространяемыми моделями.


На основании вышесказанного вам необходимо построить несколько моделей на основе нейронных сетей, позволяющих прогнозировать оптимальное торговое действие.


<b>На основе представленной информации, вам предлагается:</b>


1) Создать модель (торговую стратегию) на основе нейронных сетей для прогнозирования оптимального торгового действия. Можно использовать, как самостоятельно обученные архитектуры, так и использовать предобученные сети или фреймворки.
2) Провести тестирование разработанной стратегии на валидационном датасете.
3) Зафиксировать метрики модели для дальнейшего сравнения экспериментов.
4) Сформировать дашборд, показывающий эффективность различных торговых
стратегий.

# Подход к реализации

В рамках ДЗ №4 был подготовлен фреймворк для проведения экспериментов, логгирования их результатов и отображения в UI

Следовательно, задача на данном этапе сводится к:
- созданию нескольких классов нейросетевых моделей
- добавлению их в пул исследуемых (landing.py -> ml_model_strategy_training_loop_callback -> model_options)
- проведению тестирования с использованием имеющегося функционала

Реализацию проведу с использованием фреймворка pytorch (он мне ближе из альтернатив, а написание нейросети на чистом numpy не рассматриваю т.к. цель задания - не демонстрация понимания низкоуровневой логики)

Из архитектур - реализую CNN, LSTM, GRU как наиболее подходящие к домену прогнозирования временных рядов

# Создадим класс-обёртку для PyTorch моделей в нашем проекте

## Для облегчения разработки достанем данные в том виде, в котором они используются в training_loop.py

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

from sklearn.preprocessing import StandardScaler

from src.core import utils

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()

c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Params for testing
tickers = ["^GSPC"]
interval = config.INTERVAL

# Params for train-test-valid split
# TEST - Q4'24 | VAL - Q1'25
train_start = "2020-01-01"
train_end = "2024-10-01"
test_end = "2025-01-01"
valid_end = "2025-04-01"  # захватим первый квартал 2025, тестовая выборка по длине такая же как валидационная - 3мес

# Params inside Optimizer
early_stopping_rounds = 50
n_trials = 20


In [3]:
# Список моделей и вариантов их гиперпараметров для тестирования
# TODO: ML модели появятся здесь
model_options = {
}

In [4]:
# Часть логики training_loop - получим в блокноте те же переменные что и при инициализации optimizer.ModelOptimizer
# Get preprocessed data
data, features = utils.get_preprocessed_history(
    tickers=tickers, start=train_start, end=valid_end, interval=interval
)

# Add feature column
data = utils.add_target(data)

# Loop over tickers in dataset
grand_result = []
for ticker in data["Ticker"].unique():
    logger.info(f"~ ~ ~ Modelling for {ticker} ~ ~ ~")
    ticker_data = data[data["Ticker"] == ticker].reset_index(drop=True)

    # Split dataset into parts
    X_train, y_train, X_test, y_test, X_val, y_val = utils.train_test_valid_split(
        ticker_data,
        train_start=train_start,
        train_end=train_end,
        test_end=test_end,
        valid_end=valid_end,
        drop_leaky=True,
        target_col="target",
    )
    logger.info(
        f"{X_train.shape=} | {y_train.shape=} || {X_test.shape=} | {y_test.shape=} || {X_val.shape=} | {y_val.shape=}"
    )

    # Scale train / test / validation datasets - fit on train
    logger.info("Scaling features...")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train[features])
    X_test_scaled = scaler.transform(X_test[features])
    X_val_scaled = scaler.transform(X_val[features])

    break

    # # Для каждого потенциального типа модели:
    # for model_type, param_dict in model_options.items():
    #     logger.info(f"~ ~ Iteration for {model_type.__name__} ~ ~")
    #     # Тюним гиперпараметры на train / test датасетах, выбираем лучшее
    #     model_optimizer = optimizer.ModelOptimizer(
    #         model_type,
    #         param_dict,
    #         X_train_scaled,
    #         y_train,
    #         X_test_scaled,
    #         y_test,
    #         X_val_scaled,
    #         y_val,
    #     )
    #     (
    #         model,
    #         best_params,
    #         (train_roc_auc, test_roc_auc, val_roc_auc),
    #         (train_metrics_table, test_metrics_table, val_metrics_table),
    #     ) = model_optimizer.optimize()

[INFO   ] 2025-05-13@19:38:55: Getting preprocessed history from local cache DB...
[INFO   ] 2025-05-13@19:38:57: Got history of shape (1854, 8), 0 NaNs
[INFO   ] 2025-05-13@19:38:58: Parsed features from JSON to separate columns: (1854, 302), 0 NaNs
[INFO   ] 2025-05-13@19:38:58: Adding binary target...
[INFO   ] 2025-05-13@19:38:58: Target added: (1854, 303), 0 NaNs
[INFO   ] 2025-05-13@19:38:58: ~ ~ ~ Modelling for ^GSPC ~ ~ ~
[INFO   ] 2025-05-13@19:38:58: Splitting ticker data to train/test/validation parts
[INFO   ] 2025-05-13@19:38:58: X_train.shape=(1672, 296) | y_train.shape=(1672,) || X_test.shape=(92, 296) | y_test.shape=(92,) || X_val.shape=(90, 296) | y_val.shape=(90,)
[INFO   ] 2025-05-13@19:38:58: Scaling features...


## Заготовки внутренних функций модели

In [5]:
import numpy as np

from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
# Потенциальные параметры для тюнинга
window_size = 30
batch_size = 32
lr = 0.001
num_epochs = 10

In [7]:
def convert_to_dataloader(X_raw, y_raw=None):
    """
    Adapter to be used inside NN models
    """
    logger.info("Converting numpy arrays to TensorDataset and DataLoader...")
    # Convert to PyTorch Tensors
    X, y = [], []
    for i in range(len(X_raw) - window_size - 1):
        X.append(X_raw[i : i + window_size])
        if y_raw is not None:
            y.append(y_raw[i + window_size])
        else:
            y.append(0)

    # Now convert lists to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    # DataLoader
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader

In [8]:
def train_model(model, dataloader):
    """
    NN model training loop
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Switch model to train mode
    model.train()
    for epoch in range(num_epochs):
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    return model

In [9]:
class StockCNN(nn.Module):
    """
    Convolution Neural Network
    """
    def __init__(self, n_features, kernel_size_conv1=3, kernel_size_conv2=3):
        super(StockCNN, self).__init__()
        self.cnn_stack = nn.Sequential(
            nn.Conv1d(n_features, 16, kernel_size=kernel_size_conv1),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=kernel_size_conv2),
            nn.ReLU(),
            nn.Flatten(), # flatten after convolutions
            nn.Linear(
                32 * (window_size - (kernel_size_conv1 - 1) - (kernel_size_conv2 - 1)), 64
            ),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.Softmax(1)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Permute to (batch_size, channels, sequence_length)
        probabilities = self.cnn_stack(x)
        return probabilities
    

## Реализуем логику функций fit() и predict_proba() данных моделей

In [10]:
# (1) - to be in model's .fit() method
def fit(X_train, y_train):
    # Convert data
    dataloader = convert_to_dataloader(X_raw=X_train, y_raw=y_train)
    # Initialize model
    logger.info("Initializing model...")
    model = StockCNN(n_features=X_train.shape[1])
    # Train it
    logger.info("Training model...")
    model = train_model(model, dataloader)
    return model


In [11]:
# (2) - to be in predict_proba() method
def predict_proba(model, X_raw):
    dataloader = convert_to_dataloader(X_raw)

    _ = model.eval()
    predictions = [np.array([0.5, 0.5], dtype="float32")] * (window_size + 1) # first ticks have no real prediction
    with torch.no_grad():
        for inputs, _ in dataloader:
            logits = model(inputs)
            predictions.extend(logits.numpy())

    # Convert to a single np.array
    predictions = np.stack(predictions, axis=0)
    
    return predictions

In [12]:
# Для референса - в model_optimizer происходит следующее:
# = = = = = =
# # Инициализация модели с заданными параметрами
# model = self.model_type(**suggested_param)

# # Обучение модели с валидационной выборкой
# if self.param_dict.get("use_eval_set", None):
#     _ = model.fit(
#         self.X_train_scaled,
#         self.y_train,
#         eval_set=(self.X_test_scaled, self.y_test),
#     )
# else:
#     _ = model.fit(
#         self.X_train_scaled,
#         self.y_train,
#     )

# # Предсказания на тренировочной и тестовой выборках
# y_train_pred_prob = model.predict_proba(self.X_train_scaled)[:, 1]
# y_test_pred_prob = model.predict_proba(self.X_test_scaled)[:, 1]

In [13]:
# То что получается у нас
# fit
model = fit(X_train=X_train_scaled, y_train=y_train)
# predict
y_train_pred_prob = predict_proba(model, X_train_scaled)[:, 1]
y_test_pred_prob = predict_proba(model, X_test_scaled)[:, 1]
# log metric
train_roc_auc = roc_auc_score(y_train, y_train_pred_prob)
test_roc_auc = roc_auc_score(y_test, y_test_pred_prob)

print(f"{train_roc_auc=} | {test_roc_auc=}")


[INFO   ] 2025-05-13@19:39:21: Converting numpy arrays to TensorDataset and DataLoader...
C:\Users\Pavel\AppData\Local\Temp\ipykernel_16108\3998794152.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X = torch.tensor(X, dtype=torch.float32)
[INFO   ] 2025-05-13@19:39:23: Initializing model...
[INFO   ] 2025-05-13@19:39:23: Training model...


Epoch [1/10], Loss: 0.6659
Epoch [2/10], Loss: 0.6443
Epoch [3/10], Loss: 0.6821
Epoch [4/10], Loss: 0.5328
Epoch [5/10], Loss: 0.6582
Epoch [6/10], Loss: 0.4607
Epoch [7/10], Loss: 0.7626
Epoch [8/10], Loss: 0.5211
Epoch [9/10], Loss: 0.6265


[INFO   ] 2025-05-13@19:39:29: Converting numpy arrays to TensorDataset and DataLoader...


Epoch [10/10], Loss: 0.4940


[INFO   ] 2025-05-13@19:39:31: Converting numpy arrays to TensorDataset and DataLoader...


train_roc_auc=np.float64(0.5238771093519696) | test_roc_auc=np.float64(0.5639204545454546)


## Получив работоспособный прототип, объединяем всё в ./app/src/models/cnn_model.py и протестируем цикл подбора гиперпараметров

(запускать ячейки ниже после перезапуска ядра)

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

from src.models.training_loop import ml_model_strategy_training_loop
from src.models.cnn_model import CNNModel

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()

c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Params for testing
tickers = ["^GSPC"]
interval = config.INTERVAL

# Params for train-test-valid split
# TEST - Q4'24 | VAL - Q1'25
train_start = "2020-01-01"
train_end = "2024-10-01"
test_end = "2025-01-01"
valid_end = "2025-04-01"  # захватим первый квартал 2025, тестовая выборка по длине такая же как валидационная - 3мес

In [3]:
# # Потенциальные параметры для тюнинга
# window_size = 30
# batch_size = 32
# lr = 0.001
# num_epochs = 10
model_options = {
    CNNModel: {
        "int": {
            "window_size": {"low": 5, "high": 60},
            "batch_size": {"low": 4, "high": 128},
            "num_epochs": {"low": 5, "high": 50}
        },
        "float": {
            "lr": {"low": 0.0005, "high": 0.01}
        }
    }
}

In [4]:
result = ml_model_strategy_training_loop(
    tickers=tickers,
    interval=interval,
    train_start=train_start,
    train_end=train_end,
    test_end=test_end,
    valid_end=valid_end,
    model_options=model_options,
)

[INFO   ] 2025-05-13@20:20:59: Getting preprocessed history from local cache DB...
[INFO   ] 2025-05-13@20:21:01: Got history of shape (1854, 8), 0 NaNs
[INFO   ] 2025-05-13@20:21:02: Parsed features from JSON to separate columns: (1854, 302), 0 NaNs
[INFO   ] 2025-05-13@20:21:02: Adding binary target...
[INFO   ] 2025-05-13@20:21:02: Target added: (1854, 303), 0 NaNs
[INFO   ] 2025-05-13@20:21:02: ~ ~ ~ Modelling for ^GSPC ~ ~ ~
[INFO   ] 2025-05-13@20:21:02: Splitting ticker data to train/test/validation parts
[INFO   ] 2025-05-13@20:21:02: X_train.shape=(1672, 296) | y_train.shape=(1672,) || X_test.shape=(92, 296) | y_test.shape=(92,) || X_val.shape=(90, 296) | y_val.shape=(90,)
[INFO   ] 2025-05-13@20:21:02: Scaling features...
[INFO   ] 2025-05-13@20:21:02: ~ ~ Iteration for CNNModel ~ ~
[INFO   ] 2025-05-13@20:21:02: Searching for best hyperparameters using Optuna...
[I 2025-05-13 17:21:02,733] A new study created in memory with name: no-name-238bfaa2-4e26-497c-a786-5fdc7bfdfb0f



=== Метрики для TRAIN выборки ===
ROC AUC: 0.7388
   Cutoff  Precision     Recall   Accuracy   F1-Score
0    50.0  65.763324  81.340782  67.344498  72.727273
1    60.0  67.707317  77.541899  68.181818  72.291667
2    70.0  69.296375  72.625698  68.122010  70.921986
3    80.0  70.616114  66.592179  67.284689  68.545141

=== Метрики для TEST выборки ===
ROC AUC: 0.3840
   Cutoff  Precision     Recall   Accuracy   F1-Score
0    50.0  50.000000  87.500000  47.826087  63.636364
1    60.0  48.333333  60.416667  45.652174  53.703704
2    70.0  47.916667  47.916667  45.652174  47.916667
3    80.0  41.379310  25.000000  42.391304  31.168831

=== Метрики для VAL выборки ===
ROC AUC: 0.5012
   Cutoff  Precision     Recall   Accuracy   F1-Score
0    50.0  47.826087  76.744186  48.888889  58.928571
1    60.0  41.666667  46.511628  43.333333  43.956044
2    70.0  50.000000  27.906977  52.222222  35.820896
3    80.0  66.666667   9.302326  54.444444  16.326531


[INFO   ] 2025-05-13@20:25:17: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:19: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:20: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:21: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:21: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 2025-05-13@20:25:22: Converting numpy arrays to TensorDataset and DataLoader...
[INFO   ] 

In [5]:
result

,Experiment_ID,Ticker,Interval,Type,START_DT,END_DT,Model,Model_params,Cutoff,Precision,Recall,Accuracy,F1_Score,ROC_AUC,Return_pct,Win_Rate_pct,Num_Trades
0,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TRAIN,2020-01-01,2024-10-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",50.0,65.763324,81.340782,67.344498,72.727273,0.738796,325.101135,70.38835,206
1,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TRAIN,2020-01-01,2024-10-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",60.0,67.707317,77.541899,68.181818,72.291667,0.738796,486.080849,66.028708,209
2,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TRAIN,2020-01-01,2024-10-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",70.0,69.296375,72.625698,68.122010,70.921986,0.738796,467.10368,66.509434,212
3,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TRAIN,2020-01-01,2024-10-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",80.0,70.616114,66.592179,67.284689,68.545141,0.738796,498.668633,69.387755,196
4,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TEST,2024-10-01,2025-01-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",50.0,50.000000,87.500000,47.826087,63.636364,0.383996,1.777532,71.428571,7
5,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TEST,2024-10-01,2025-01-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",60.0,48.333333,60.416667,45.652174,53.703704,0.383996,-4.156214,33.333333,12
6,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TEST,2024-10-01,2025-01-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",70.0,47.916667,47.916667,45.652174,47.916667,0.383996,-4.256129,43.75,16
7,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,TEST,2024-10-01,2025-01-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",80.0,41.379310,25.000000,42.391304,31.168831,0.383996,-11.165178,50.0,18
8,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,VALID,2025-01-01,2025-04-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",50.0,47.826087,76.744186,48.888889,58.928571,0.501237,-6.225919,66.666667,3
9,17307833-3006-11f0-959c-240a64112db6,^GSPC,1d,VALID,2025-01-01,2025-04-01,CNNModel,"{""window_size"": 16, ""batch_size"": 98, ""num_epo...",60.0,41.666667,46.511628,43.333333,43.956044,0.501237,-8.535883,66.666667,6


In [ ]:
# см блокноты "CNN LSTM GRU" и практику из 24го урока
# возможно добавить Transformer из 25го урока?
